In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [23]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/cities2.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Nikolskoye,RU,8/22/2020 1:55,59.70,30.79,57.20,71,96,4.47
1,1,Bajil,YE,8/22/2020 1:57,15.06,43.29,88.48,62,83,8.08
2,2,Albany,US,8/22/2020 1:56,42.60,-73.97,75.00,89,95,3.80
3,3,Mar Del Plata,AR,8/22/2020 1:54,-38.00,-57.56,50.00,93,0,2.24
4,4,Los Llanos De Aridane,ES,8/22/2020 1:56,28.66,-17.92,73.40,83,20,12.75


In [24]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 85


In [25]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Hilo,US,8/22/2020 1:56,19.73,-155.09,84.20,62,1,10.29
20,20,Kapaa,US,8/22/2020 1:52,22.08,-159.32,82.40,69,90,11.41
26,26,Port Blair,IN,8/22/2020 1:56,11.67,92.75,83.57,77,100,9.40
32,32,Butaritari,KI,8/22/2020 1:56,3.07,172.79,82.24,65,100,2.59
33,33,Rawlins,US,8/22/2020 1:57,41.79,-107.24,82.40,12,1,11.41
43,43,Bambanglipuro,ID,8/22/2020 1:57,-7.95,110.28,83.34,65,29,7.38
46,46,Isangel,VU,8/22/2020 1:56,-19.55,169.27,82.40,78,40,9.17
50,50,Bol,TD,8/22/2020 1:57,13.46,14.71,80.89,74,100,7.07
51,51,Vaini,TO,8/22/2020 1:56,-21.20,-175.20,80.60,83,40,6.93
52,52,Tamiahua,MX,8/22/2020 1:53,21.27,-97.45,81.25,82,0,8.95


In [26]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID       89
City          89
Country       89
Date          89
Lat           89
Lng           89
Max Temp      89
Humidity      89
Cloudiness    89
Wind Speed    89
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID       89
City          89
Country       89
Date          89
Lat           89
Lng           89
Max Temp      89
Humidity      89
Cloudiness    89
Wind Speed    89
dtype: int64

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp",  "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Hilo,US,84.20,19.73,-155.09,
20,Kapaa,US,82.40,22.08,-159.32,
26,Port Blair,IN,83.57,11.67,92.75,
32,Butaritari,KI,82.24,3.07,172.79,
33,Rawlins,US,82.40,41.79,-107.24,
43,Bambanglipuro,ID,83.34,-7.95,110.28,
46,Isangel,VU,82.40,-19.55,169.27,
50,Bol,TD,80.89,13.46,14.71,
51,Vaini,TO,80.60,-21.20,-175.20,
52,Tamiahua,MX,81.25,21.27,-97.45,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass
        

Hotel not found... skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Hilo,US,84.20,19.73,-155.09,Hilo Hawaiian Hotel
20,Kapaa,US,82.40,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
26,Port Blair,IN,83.57,11.67,92.75,Fortune Resort Bay Island
32,Butaritari,KI,82.24,3.07,172.79,Isles Sunset Lodge
33,Rawlins,US,82.40,41.79,-107.24,Holiday Inn Express Rawlins
...,...,...,...,...,...,...
545,Neiafu,TO,82.40,-18.65,-173.98,Mystic Sands
554,Bathsheba,BB,84.20,13.22,-59.52,Atlantis Hotel
564,Bengkulu,ID,83.07,-3.80,102.27,Grage Hotel Bengkulu
571,Hopelchen,MX,83.25,19.77,-89.85,Hotel Jaguar


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
max_temp = hotel_df["Max Temp"]
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))